In [1]:
import mlflow
from pyspark.sql import SparkSession

from src.config import ProjectConfig, Tags
from src.airline_reviews.models.basic_model import BasicModel

In [2]:
config = ProjectConfig.from_yaml(config_path="../project_config.yml")
spark = SparkSession.builder.getOrCreate()
tags = Tags(**{"git_sha": "9e66454", "branch": "efehan_week2"})

In [3]:
config

ProjectConfig(num_features=['seat_comfort', 'cabin_service', 'food_bev', 'entertainment', 'ground_service', 'value_for_money'], cat_features=['traveller_type', 'cabin'], target='recommended', data_path='/Volumes/mlops_dev/efedanis/data/airline_reviews.csv', catalog_name='mlops_dev', schema_name='efedanis', experiment_name_basic='/Shared/airline-reviews-basic', parameters={'learning_rate': 0.01, 'n_estimators': 1000, 'max_depth': 6})

In [4]:
basic_model = BasicModel(config=config, tags=tags, spark=spark)

In [5]:
basic_model.load_data()
basic_model.prepare_features()

2025-02-15 14:20:53.786 | INFO     | src.airline_reviews.models.basic_model:load_data:46 - 🔄 Loading data from Databricks tables...
2025-02-15 14:20:57.871 | INFO     | src.airline_reviews.models.basic_model:load_data:56 - ✅ Data successfully loaded.
2025-02-15 14:20:57.872 | INFO     | src.airline_reviews.models.basic_model:prepare_features:66 - 🔄 Defining preprocessing pipeline...
2025-02-15 14:20:57.872 | INFO     | src.airline_reviews.models.basic_model:prepare_features:74 - ✅ Preprocessing pipeline defined.


In [6]:
basic_model.train()
basic_model.log_model()

2025-02-15 14:20:57.883 | INFO     | src.airline_reviews.models.basic_model:train:80 - 🚀 Starting training...


[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 0, number of negative: 20000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 52
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000000 -> initscore=-34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

2025-02-15 14:21:06.170 | INFO     | src.airline_reviews.models.basic_model:log_model:99 - 📊 Mean Squared Error: 1.0
2025-02-15 14:21:06.173 | INFO     | src.airline_reviews.models.basic_model:log_model:100 - 📊 Mean Absolute Error: 1.0
2025-02-15 14:21:06.173 | INFO     | src.airline_reviews.models.basic_model:log_model:101 - 📊 F1 Score: 1.0
c:\Users\90533\marvelous-databricks-course-efehandanisman\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With M

In [12]:
run_id = mlflow.search_runs(
    experiment_names=["/Shared/airline-reviews-basic"], filter_string="tags.branch='efehan_week2'"
).run_id[0]

model = mlflow.sklearn.load_model(f"runs:/{run_id}/lightgbm-pipeline-model")

In [13]:
basic_model.retrieve_current_run_dataset()


2025-02-15 14:21:40.794 | INFO     | src.airline_reviews.models.basic_model:retrieve_current_run_dataset:150 - ✅ Dataset source loaded.


,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,overall,airline,traveller_type,cabin,recommended,unique_id,update_timestamp_utc
0,None,None,None,None,None,1,1,Emirates,Family Leisure,Economy Class,0,3384,2025-02-15 09:25:05.414
1,None,None,None,None,None,1,1,Turkish Airlines,Business,Economy Class,0,327,2025-02-15 09:25:05.414
2,None,None,None,None,None,1,1,Turkish Airlines,Family Leisure,Economy Class,0,533,2025-02-15 09:25:05.414
3,None,None,None,None,None,1,1,Turkish Airlines,Family Leisure,Economy Class,0,760,2025-02-15 09:25:05.414
4,None,None,None,None,None,1,1,Turkish Airlines,Solo Leisure,Economy Class,0,394,2025-02-15 09:25:05.414
5,None,None,None,None,None,1,2,Emirates,Solo Leisure,Economy Class,0,3039,2025-02-15 09:25:05.414
6,None,None,None,None,1,1,1,Emirates,Couple Leisure,Economy Class,0,2996,2025-02-15 09:25:05.414
7,None,None,None,None,1,1,1,Emirates,Solo Leisure,Economy Class,0,3054,2025-02-15 09:25:05.414
8,None,None,None,None,1,1,1,Emirates,Solo Leisure,Economy Class,0,3508,2025-02-15 09:25:05.414
9,None,None,None,None,1,1,1,Qatar Airways,Business,Economy Class,0,1801,2025-02-15 09:25:05.414


In [14]:
basic_model.retrieve_current_run_metadata()


2025-02-15 14:21:43.793 | INFO     | src.airline_reviews.models.basic_model:retrieve_current_run_metadata:160 - ✅ Dataset metadata loaded.


({'accuracy': 1.0, 'f1_score': 1.0, 'precision': 1.0},
 {'learning_rate': '0.01',
  'max_depth': '6',
  'model_type': 'LightGBM with preprocessing',
  'n_estimators': '1000'})

In [15]:
basic_model.register_model()

2025-02-15 14:21:43.802 | INFO     | src.airline_reviews.models.basic_model:register_model:126 - 🔄 Registering the model in UC...
Registered model 'mlops_dev.efedanis.airline_reviews_basic' already exists. Creating a new version of this model...
Created version '3' of model 'mlops_dev.efedanis.airline_reviews_basic'.
2025-02-15 14:21:44.336 | INFO     | src.airline_reviews.models.basic_model:register_model:132 - ✅ Model registered as version 3.


In [16]:
test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").limit(10)

X_test = test_set.drop(config.target).toPandas()

predictions_df = basic_model.load_latest_model_and_predict(X_test)

2025-02-15 14:21:46.611 | INFO     | src.airline_reviews.models.basic_model:load_latest_model_and_predict:171 - 🔄 Loading model from MLflow alias 'production'...
2025-02-15 14:21:47.312 | INFO     | src.airline_reviews.models.basic_model:load_latest_model_and_predict:176 - ✅ Model successfully loaded.
